<a href="https://colab.research.google.com/github/Nagmashaik123/MedConcepts/blob/main/TweetFormatted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import json
import csv
import collections

finalList = []
def writeTweetData():
    dyfunctionfile = open('MedicalDyfunctions.json')
    jsonList = json.load(dyfunctionfile)
    global dataList
    # fileName = 'MedicalConcepts'
    tweetDictionary = {}
    fileName='MedicalConcepts'
    with open('%s.csv' % (fileName), 'w') as file:
        w = csv.writer(file)
        w.writerow(['CUID', 'Concept', 'AuthorID', 'TweetText', 'CreatedDate', 'HashTags', 'Language'])
        for eachJson in jsonList:
            cui = eachJson['cui']
            concept = eachJson['concepts']
            tweet_data = eachJson['tweets']['data']
            for each_tweet_data in tweet_data:
                author_id = each_tweet_data['author_id']
                text = each_tweet_data['text']
                lang = each_tweet_data['lang']
                created_at = each_tweet_data['created_at']
                tweet_data_dic = {
                 "CUID": cui,
                 "Concept": concept,
                 "AuthorID": author_id,
                 "TweetText": text,
                 "Language":lang,
                 "CreatedDate": created_at
                 }
                if 'en' not in lang:
                    pass
                else:
                    if len(finalList) != 0:
                        exists = is_tweet_exists(finalList,tweet_data_dic)
                        if not exists:
                            hashTags = get_hashtags(text)
                            if hashTags is not None:
                                tweet_data_dic["HashTags"]=hashTags
                            else:
                                tweet_data_dic["HashTags"]=''
                            finalList.append(tweet_data_dic)
                            w.writerow([tweet_data_dic['CUID'], tweet_data_dic['Concept'],tweet_data_dic['AuthorID'],tweet_data_dic['TweetText'],tweet_data_dic['CreatedDate'],tweet_data_dic['HashTags'], tweet_data_dic['Language']])

                    else:
                        hashTags = get_hashtags(text)
                        if hashTags is not None:
                            tweet_data_dic["HashTags"]=hashTags
                        else:
                            tweet_data_dic["HashTags"]=''
                        finalList.append(tweet_data_dic)
                        w.writerow([tweet_data_dic['CUID'], tweet_data_dic['Concept'],tweet_data_dic['AuthorID'],tweet_data_dic['TweetText'],tweet_data_dic['CreatedDate'],tweet_data_dic['HashTags'], tweet_data_dic['Language']])



def is_tweet_exists(finalList, tweet_data_dic):
    for dict in finalList:
        if (tweet_data_dic['CUID'] == dict['CUID']) and (tweet_data_dic['AuthorID'] == dict['AuthorID']) and (tweet_data_dic['TweetText'] == dict['TweetText']) and (tweet_data_dic['CreatedDate'] == dict['CreatedDate']):
            return True
    return False


def get_hashtags(text):
    hashTagSet=set()
    hashtag_list = []
    hashtags = ''
    if '#' in text:
        for word in text.split():
            if word[0] == '#':
                hashtag_list.append(word[1:])
                hashTagSet= set(hashtag_list)
                hashtags = ', '.join(hashTagSet)
        return hashtags
    return None

writeTweetData()



In [76]:
import pandas as pd
df = pd.read_csv(r"MedicalConcepts.csv")

df



,CUID,Concept,AuthorID,TweetText,CreatedDate,HashTags,Language
0,C0003123,Anorexia,1399574560601673728,switch the v and the r and even Instagram ads ...,2021-06-25T23:44:49.000Z,"thinspo, anorexia",en
1,C0003123,Anorexia,1347182874986950658,here use me as thinspo 🦋\n\n#thinspo #proana #...,2021-06-25T23:41:26.000Z,"starvingforana, eatingdisorder., anorexic, edt...",en
2,C0003123,Anorexia,1345332202847019008,RT @starvingd0lly: ♡ some of my favorite thins...,2021-06-25T21:42:02.000Z,"anorexic, anorexia, thinspothread, thinsp0, pr...",en
3,C0003123,Anorexia,1370637609861455876,Im doing a 16 hour fast and only have two hour...,2021-06-25T21:22:19.000Z,"thinsp0, anorexia, proanatwt",en
4,C0003123,Anorexia,709854932644597760,Losing weight are hunger um why #anorexia,2021-06-25T21:19:52.000Z,anorexia,en
...,...,...,...,...,...,...,...
19037,C0231230,Fatigability,2266887595,http://t.co/z1XYcLiOHW Get your own NSA EMAIL ...,2014-09-01T02:45:37.000Z,"NSAorg, fatigability, springily",en
19038,C0231230,Fatigability,2195306756,RT @USMLE_CKHints: #Myasthenia gravis The typ...,2013-12-31T03:44:28.000Z,"Myasthenia, fatigability",en
19039,C0231230,Fatigability,382975437,#Myasthenia gravis The typical presentation i...,2013-12-31T03:39:18.000Z,"Myasthenia, fatigability",en
19040,C0231230,Fatigability,152584587,Tired of doing this crappy sh*t. :p 😝😂 #wasted...,2013-02-09T12:28:41.000Z,"instapic, fatigability, lol, wasted",en


In [77]:
import re

# Reading emoji patterns which contains emoticons , symbols & pictographs, transport & map symbols , flags
emojisCollection = re.compile("["
                       u"\U0001F600-\U0001F64F"  
                       u"\U0001F300-\U0001F5FF"  
                       u"\U0001F680-\U0001F6FF"  
                       u"\U0001F1E0-\U0001F1FF"  
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
df['CleanedTweet'] = (df['TweetText'].str.lower().str.replace(r'[^\w\s]+', '').str.replace(emojisCollection, '').str.strip())
df['CleanHashTags'] = (df['HashTags'].str.lower().str.replace(r'[^\w\s]+', ',').str.replace(emojisCollection, '').str.strip())

In [78]:
dataFrame=df[['CUID','Concept','AuthorID','TweetText','CleanedTweet','CreatedDate','HashTags','CleanHashTags','Language']]
dataFrame

,CUID,Concept,AuthorID,TweetText,CleanedTweet,CreatedDate,HashTags,CleanHashTags,Language
0,C0003123,Anorexia,1399574560601673728,switch the v and the r and even Instagram ads ...,switch the v and the r and even instagram ads ...,2021-06-25T23:44:49.000Z,"thinspo, anorexia","thinspo, anorexia",en
1,C0003123,Anorexia,1347182874986950658,here use me as thinspo 🦋\n\n#thinspo #proana #...,here use me as thinspo \n\nthinspo proana anor...,2021-06-25T23:41:26.000Z,"starvingforana, eatingdisorder., anorexic, edt...","starvingforana, eatingdisorder, anorexic, edtw...",en
2,C0003123,Anorexia,1345332202847019008,RT @starvingd0lly: ♡ some of my favorite thins...,rt starvingd0lly some of my favorite thinspos...,2021-06-25T21:42:02.000Z,"anorexic, anorexia, thinspothread, thinsp0, pr...","anorexic, anorexia, thinspothread, thinsp0, pr...",en
3,C0003123,Anorexia,1370637609861455876,Im doing a 16 hour fast and only have two hour...,im doing a 16 hour fast and only have two hour...,2021-06-25T21:22:19.000Z,"thinsp0, anorexia, proanatwt","thinsp0, anorexia, proanatwt",en
4,C0003123,Anorexia,709854932644597760,Losing weight are hunger um why #anorexia,losing weight are hunger um why anorexia,2021-06-25T21:19:52.000Z,anorexia,anorexia,en
...,...,...,...,...,...,...,...,...,...
19037,C0231230,Fatigability,2266887595,http://t.co/z1XYcLiOHW Get your own NSA EMAIL ...,httptcoz1xycliohw get your own nsa email addre...,2014-09-01T02:45:37.000Z,"NSAorg, fatigability, springily","nsaorg, fatigability, springily",en
19038,C0231230,Fatigability,2195306756,RT @USMLE_CKHints: #Myasthenia gravis The typ...,rt usmle_ckhints myasthenia gravis the typica...,2013-12-31T03:44:28.000Z,"Myasthenia, fatigability","myasthenia, fatigability",en
19039,C0231230,Fatigability,382975437,#Myasthenia gravis The typical presentation i...,myasthenia gravis the typical presentation in...,2013-12-31T03:39:18.000Z,"Myasthenia, fatigability","myasthenia, fatigability",en
19040,C0231230,Fatigability,152584587,Tired of doing this crappy sh*t. :p 😝😂 #wasted...,tired of doing this crappy sht p wasted fatig...,2013-02-09T12:28:41.000Z,"instapic, fatigability, lol, wasted","instapic, fatigability, lol, wasted",en


In [79]:
df_new = df[df['CleanHashTags'].notnull()]
conceptFrequency = {}
def freqCount():
  for hashtag in df_new["CleanHashTags"]:
    strToken = hashtag.split(', ')
    for token in strToken:
      if token not in conceptFrequency.keys():
        conceptFrequency[token] = 1
      else:
        conceptFrequency[token] = conceptFrequency[token]+1
  print(conceptFrequency)
  return conceptFrequency

freqCount()

df = pd.DataFrame(list(conceptFrequency.items()),columns = ['Concept','Frequency']) 
df= df.sort_values(by='Frequency', ascending=False, ignore_index=True)
df

{'thinspo': 82, 'anorexia': 494, 'starvingforana': 5, 'eatingdisorder': 102, 'anorexic': 34, 'edtw': 6, 'boneybody': 2, 'skinnyteen': 4, 'starving': 7, 'ed': 82, 'skinnygirl': 5, 'probone': 6, 'skinnylove': 4, 'proana': 93, 'boneylegs': 2, 'bone': 6, 'thinlegs': 3, 'edtwt': 170, 'thinthighs': 3, 'bonespo': 14, 'thinspothread': 11, 'thinsp0': 39, 'proanatwt': 46, 'proa': 1, 'proanacoach': 8, 'thinspi': 1, 'edtwtdiet': 5, 'harshedtwt': 1, 'edtwitter': 5, 'edtwtthread': 4, 'eatingdisorders': 29, 'recoveryispossible': 2, 'thepaincoach': 6, 'chronicpain': 496, 'hardspo': 1, 'meanspo': 27, 'softspo': 1, 'edrecovery': 24, 'hospital': 19, 'anorexiarecovery': 5, 'karencarpenter': 3, 'anorexia,': 9, 'shtwt': 7, 'fasting': 9, 'edtwtpoll': 3, '3dtwt': 1, 'styrotwt': 1, 'binge': 3, 'edtwtthreads': 1, 'miatwt': 3, 'ana': 50, 'nsfwtwt': 1, 'binging': 2, 'bed': 8, 'bulimia': 65, 'sweetspo': 13, 'fast': 7, 'mia': 14, 'anatwt': 14, 'recovery': 100, 'therapists': 1, 'speaker': 3, 'training': 10, 'therapy

,Concept,Frequency
0,headache,902
1,fibromyalgia,776
2,sleeplessness,725
3,bruxism,722
4,eczema,721
...,...,...
14565,bikelanes,1
14566,dutchreach,1
14567,righthook,1
14568,bikesafety,1
